In [2]:
import requests
import json
import csv

FILENAME ='SeasonToBrandMapping-RohinisPlus.csv'
customerTypesList = ['BACKBOOK_NO_CINEMA','FRONTBOOK_OR_DEFAULT']
discoveryFeedRequestUrl = 'https://contentdata.vision.bt.com/cptmdss/V7.0/discoveryFeed'

brandRequestUrl = 'https://contentdata.vision.bt.com/cptmdss/V7.0/brandDetails'
brandRequestQueryParams = {
    'context': 'commendaroo-testing',
}

        
def callDiscoveryFeed(feedUrl, queryParams):
    request = requests.get(feedUrl, params=queryParams)    
    return request.json()["discoveryFeedResponse"]["items"]

def getSeasonsForBrand(brandGuid):
    brandRequestQueryParams["brandGuid"] = brandGuid
    request = requests.get(brandRequestUrl, params=brandRequestQueryParams)
    groups = request.json()["bdResponse"]["brandInfo"]["groups"]
    seasonGuids = []
    for season in groups:
        seasonGuids.append(season["guid"].encode("utf-8"))
    return seasonGuids

def getBrandListForFeedUrl(feedUrl, queryParams, brandGuidList, existingBrandGuids):
    queryParamsForCall = queryParams
    gotContent = True
    while gotContent == True:
        itemList = callDiscoveryFeed(feedUrl, queryParamsForCall)   ## Get items for this page (because response is paged)
        if (len(itemList) == 0):                                    ## If page had no items, we've reached the bottom of the available content
            gotContent = False                                      ##If the call returns nothing then we've got no data to work with so break
            continue
                
        for content in itemList:                                    ## If we have content, loop through each one
            if (content['type']) == 'BRAND':                        ## Select only brand contents
                if(content["guid"] in brandGuidList):               ## Check if we've found this brandGuid before on a previous call
                    continue                                        
                brandGuidList.append(content["guid"].encode("utf-8"))## If not pop it in the list

        queryParamsForCall['cursor'] = itemList[-1]["guid"]         ## Set us up for the next loop, telling it to use that last content as the cursor location
        queryParamsForCall['direction'] = 'FORWARD'

def fillExistingArrays(existingSeasonToBrand, existingBrandGuids):
    
    reader = pd.read_csv('s3://bt-data-science-playground/bt-tv-recommendation-system/model_objects/New-SeasonToBrandMapping.csv')
    for index, row in reader.iterrows():
        if(row['BRAND_GUID'] in existingSeasonToBrand):
            existingSeasonToBrand[row['BRAND_GUID']].append(row['SEASON_GUID'])
        else:
            existingSeasonToBrand[row['BRAND_GUID']] = [row['SEASON_GUID']]
        if(row['BRAND_GUID'] not in existingBrandGuids):
            existingBrandGuids.append(row['BRAND_GUID'])

def writeToCsv(arrayToWrite):
    with open(FILENAME, 'ab') as csvfile:
        fieldnames = ['SEASON_GUID', 'BRAND_GUID']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        for item in arrayToWrite:
            writer.writerow(item)


############################
######### Main Run #########
############################
existingSeasonToBrand = {}
existingBrandGuids = []
seasonToBrand = []
brandGuidList = []
fillExistingArrays(existingSeasonToBrand,existingBrandGuids)

############################
######## Get Brands ########
############################
tags = ['TV', 'TV Replay', 'Kids']
for tag in tags:
    for customerType in customerTypesList:
        queryParams = {
            'byTags': '{schedulerChannel, ' + tag + '}',
            'context': 'commendaroo-testing',
        }
        queryParams['customerType'] = customerType
        getBrandListForFeedUrl(discoveryFeedRequestUrl, queryParams, brandGuidList, existingBrandGuids)
        print tag + 'BrandList + ' + customerType + ' full brand list len : ' + str(len(brandGuidList))

print len(brandGuidList)

############################
####### Get Seasons ########
############################
for brandGuid in brandGuidList:
    ## Check if this brandGuid exists already, if not add it, if it does check the seasonGuid
    if (brandGuid not in existingSeasonToBrand):
        for seasonGuid in getSeasonsForBrand(brandGuid):
            ## Add these to our dictionary
            seasonToBrand.append({
                "SEASON_GUID" : seasonGuid,
                "BRAND_GUID" : brandGuid
            })
    else:
        for seasonGuid in getSeasonsForBrand(brandGuid):
            ## Check if the seasonguid is already recorded for this brand
            if (seasonGuid not in existingSeasonToBrand[brandGuid]):
                ## Add these to our dictionary
                seasonToBrand.append({
                    "SEASON_GUID" : seasonGuid,
                    "BRAND_GUID" : brandGuid
                })

print len(seasonToBrand)
print seasonToBrand

############################
####### Write to Csv #######
############################
writeToCsv(seasonToBrand)

TVBrandList + BACKBOOK_NO_CINEMA full brand list len : 389
TVBrandList + FRONTBOOK_OR_DEFAULT full brand list len : 389
TV ReplayBrandList + BACKBOOK_NO_CINEMA full brand list len : 653
TV ReplayBrandList + FRONTBOOK_OR_DEFAULT full brand list len : 653
KidsBrandList + BACKBOOK_NO_CINEMA full brand list len : 755
KidsBrandList + FRONTBOOK_OR_DEFAULT full brand list len : 755
755
0
[]
